In [ ]:
%%capture

from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

%run s05_ai_engine.ipynb

# Dirty hack to import types from other file for static analysis tools
try:
    from . import ScoredMove, IterativeAlphaBetaCached, HumanEngine, playGame, middlegame_board, result_iterativeAlphaBetaCached, cache_alpha_beta, NodeType
    import chess
    import random
except ImportError:
    pass

# Enhanced AI Engine

In diesem Abschnitt geht es darum auf dem bisherigen Alpha-Beta Algorithmus und der ersten implementierten Evaluierungsfunktion aufzubauen, um die Spielstärke der Engine weiter zu steigern.

## Simple Evaluation Function

Zunächst soll die bisher eingesetzte Evaluierungsfunktion, die lediglich die Werte der Figuren betrachtet, durch die komplexere [Simplified Evaluation Function](https://www.chessprogramming.org/Simplified_Evaluation_Function) ersetzt werden. Auf Grund der höheren Komplexität wird diese nun im Gegensatz zu vorher als eine eigene Klasse `SimplifiedEvaluation` implementiert. 

In [ ]:
class SimplifiedEvaluation():
    pass

Auch die Simplified Evaluation Function weist den einzelnen Figuren Werte zu. Diese sind sehr ähnlich zu den vorherigen Werten von Capablanca, werden jedoch nun in Centipawns ausgedrückt. Konkret wurde eine Reihe von Bedingungen festgelegt, die auf eigener Erfahrung des Autors basieren. Beispielsweise wird der Läufer nun etwas besser angesehen als der Springer, da dadurch die Engine versucht selbst das Läuferpaar zu behalten und das des Gegners zu nehmen. Das Dictionary `PIECE_VALUES` enthält nachfolgend eine mögliche Wertebelegung, welche die aufgestellen Bedingungen erfüllen.

In [ ]:
PIECE_VALUES = {
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 20000
}

SimplifiedEvaluation.PIECE_VALUES = PIECE_VALUES

Ein Schachmatt wird erneut mit einem so hohen Wert besetzt, dass dieser auf keinem anderem Wege erreicht werden kann und das Remis wieder mit 0.

In [ ]:
SimplifiedEvaluation.VALUE_CHECKMATE = 30000
SimplifiedEvaluation.VALUE_DRAW = 0

Die zweite Idee, auf der die Simplified Evaluation Engine aufbaut, sind [Piece Square Tables](https://www.chessprogramming.org/Piece-Square_Tables). Damit die Engine ihre Figuren strategisch sinnvoll platziert, werden gut platzierte Figuren zusätzlich belohnt und schlecht platzierte Figuren bestraft. Dies wird technisch realisiert, indem für jeden Figurentyp jedes Feld auf dem Schachbrett mit einem spezifischen Wert belegt ist. `PAWN_SQUARE_VALUES` zeigt die Werte der Schachfelder als 2 dimensionale 8 x 8 Liste für den weißen Bauern. Der erste Eintrag entspricht dabei der 8. Reihe auf dem Schachfeld und der letze Eintrag der ersten Reihe. Zu sehen ist dabei beispielsweise, dass Bauern auf der siebten Reihe einen hohen Bonus mit 50 erhalten, da diese kurz vor einer Promotion stehen. 

In [ ]:
PAWN_SQUARE_VALUES = [
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [50, 50, 50, 50, 50, 50, 50, 50],
    [10, 10, 20, 30, 30, 20, 10, 10],
    [ 5,  5, 10, 25, 25, 10,  5,  5],
    [ 0,  0,  0, 20, 20,  0,  0,  0],
    [ 5, -5,-10,  0,  0,-10, -5,  5],
    [ 5, 10, 10,-20,-20, 10, 10,  5],
    [ 0,  0,  0,  0,  0,  0,  0,  0]
] # yapf: disable

Um die Berechnung später zu vereinfachen können die oben definierte Figurenwerte direkt als Offset mit in die Tabelle geschrieben werden. Dies übernimmt die Hilfsfunktion `add_offset_to_square_values`, welche eine piece square table und einen offset als Parameter entgegennimmt und dann den Offset zu jedem Eintrag der Tabelle hinzuaddiert.

In [ ]:
def add_offset_to_square_values(square_values: list[list[int]], offset: int):
    for row in range(8):
        for column in range(8):
            square_values[row][column] += offset

Für die `PAWN_SQUARE_VALUES` wird entsprechend als Offset der Figurenwert des Bauern aus dem `PIECE_VALUES` Dictionary hinzugefügt.

In [ ]:
add_offset_to_square_values(PAWN_SQUARE_VALUES, PIECE_VALUES[chess.PAWN])

Analog wird dies anschließend auch alle weiteren Tabellen für die jeweiligen Figuren definiert. Teilweise werden für Schwarze und Weiße Figuren verschiedene Tabellen definiert, in dem Fall der `Simplified Evaluation Function` funktionieren die Tabellen jedoch für beide Seiten.

In [ ]:
KNIGHT_SQUARE_VALUES = [
    [-50,-40,-30,-30,-30,-30,-40,-50],
    [-40,-20,  0,  0,  0,  0,-20,-40],
    [-30,  0, 10, 15, 15, 10,  0,-30],
    [-30,  5, 15, 20, 20, 15,  5,-30],
    [-30,  0, 15, 20, 20, 15,  0,-30],
    [-30,  5, 10, 15, 15, 10,  5,-30],
    [-40,-20,  0,  5,  5,  0,-20,-40],
    [-50,-40,-30,-30,-30,-30,-40,-50]
]  # yapf: disable

add_offset_to_square_values(KNIGHT_SQUARE_VALUES, PIECE_VALUES[chess.KNIGHT])

In [ ]:
BISHOP_SQUARE_VALUES = [
    [-20,-10,-10,-10,-10,-10,-10,-20],
    [-10,  0,  0,  0,  0,  0,  0,-10],
    [-10,  0,  5, 10, 10,  5,  0,-10],
    [-10,  5,  5, 10, 10,  5,  5,-10],
    [-10,  0, 10, 10, 10, 10,  0,-10],
    [-10, 10, 10, 10, 10, 10, 10,-10],
    [-10,  5,  0,  0,  0,  0,  5,-10],
    [-20,-10,-10,-10,-10,-10,-10,-20]
]  # yapf: disable

add_offset_to_square_values(BISHOP_SQUARE_VALUES, PIECE_VALUES[chess.BISHOP])

In [ ]:
ROOK_SQUARE_VALUES = [
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 5, 10, 10, 10, 10, 10, 10,  5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [ 0,  0,  0,  5,  5,  0,  0,  0]
] # yapf: disable

add_offset_to_square_values(ROOK_SQUARE_VALUES, PIECE_VALUES[chess.ROOK])

In [ ]:
QUEEN_SQUARE_VALUES = [
    [-20,-10,-10, -5, -5,-10,-10,-20],
    [-10,  0,  0,  0,  0,  0,  0,-10],
    [-10,  0,  5,  5,  5,  5,  0,-10],
    [ -5,  0,  5,  5,  5,  5,  0, -5],
    [  0,  0,  5,  5,  5,  5,  0, -5],
    [-10,  5,  5,  5,  5,  5,  0,-10],
    [-10,  0,  5,  0,  0,  0,  0,-10],
    [-20,-10,-10, -5, -5,-10,-10,-20]
] # yapf: disable

add_offset_to_square_values(QUEEN_SQUARE_VALUES, PIECE_VALUES[chess.QUEEN])

In [ ]:
KING_SQUARE_VALUES = [
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-20,-30,-30,-40,-40,-30,-30,-20],
    [-10,-20,-20,-20,-20,-20,-20,-10],
    [ 20, 20,  0,  0,  0,  0, 20, 20],
    [ 20, 30, 10,  0,  0, 10, 30, 20],
]# yapf: disable

add_offset_to_square_values(KING_SQUARE_VALUES, PIECE_VALUES[chess.KING])

Nachdem alle piece square tables definiert wurden, kann nun auch hier analog zu den Figurenwerten ein Dictionary `PIECE_SQUARE_VALUES` erstellt werden, welches jeder Figur die entsprechende Tabelle zuweist.

In [ ]:
# TODO: Add support for endgame square table (for instance for the king)
# Idea: Two tables (beginning and end) -> interpolate (https://www.chessprogramming.org/Piece-Square_Tables#Multiple_Tables)
PIECE_SQUARE_VALUES = {
    chess.PAWN: PAWN_SQUARE_VALUES,
    chess.KNIGHT: KNIGHT_SQUARE_VALUES,
    chess.BISHOP: BISHOP_SQUARE_VALUES,
    chess.ROOK: ROOK_SQUARE_VALUES,
    chess.QUEEN: QUEEN_SQUARE_VALUES,
    chess.KING: KING_SQUARE_VALUES
}
SimplifiedEvaluation.PIECE_SQUARE_VALUES = PIECE_SQUARE_VALUES

Die Funktion `_get_piece_square_value` nimmt eine Schachfigur `piece` und deren Position mit `file` und `rank` entgegen und gibt den entsprechenden Wert aus der piece square table zurück. Für die Weiße Seite muss die Tabelle zur x-achse gespiegelt werden, entsprechend wird dort mit `7 - rank` drauf zugegriffen.

In [ ]:
def _get_piece_square_value(self, piece: chess.Piece, file: int, rank: int):
    if piece.color == chess.WHITE:
        return self.PIECE_SQUARE_VALUES[piece.piece_type][7 - rank][file]
    else:
        return self.PIECE_SQUARE_VALUES[piece.piece_type][rank][file]


SimplifiedEvaluation._get_piece_square_value = _get_piece_square_value

Da häufig über Schachfelder iteriert werden muss, wird zusätzlich noch eine Funktion `_get_square_value` implementiert, welche das Board `board` und ein Schachfeld `square` entgegennimmt und daraus die Figur, file und rank berechnen und an die zuvor definierte Methode `_get_square_value` weiterleitet.

In [ ]:
def _get_square_value(self, board: chess.Board, square: chess.Square):
    piece = board.piece_at(square)
    file = chess.square_file(square)
    rank = chess.square_rank(square)
    return self._get_piece_square_value(piece, file, rank)


SimplifiedEvaluation._get_square_value = _get_square_value

Nun kann eine absolute Heuristik `absolute_heuristic` definiert. Sie nimmt als Parameter ein Schachbrett `board` entgegen und gibt die aktuelle Bewertung aus Sicht des Spielers am Zug zurück. Dazu iteriert sie zunächst über alle besetzen Felder des Spielers am Zug und summiert die Werte für diese Felder. Anschließend werden die Werte der gegnerischen besetzen Felder abgezogen.

In [ ]:
def absolute_heuristic(self, board: chess.Board) -> int:
    """Calculate the value of all pieces on the board"""
    score = 0

    for square in chess.scan_reversed(board.occupied_co[board.turn]):
        score += self._get_square_value(board, square)

    for square in chess.scan_reversed(board.occupied_co[not board.turn]):
        score -= self._get_square_value(board, square)

    return score


SimplifiedEvaluation.absolute_heuristic = absolute_heuristic

Auch die bereits bekannte `evaluate` Funktion wird der `SimplifiedEvaluation` Klasse hinzugefügt, da diese die benötigten Werte `VALUE_CHECKMATE` und `VALUE_DRAW` hat.

In [ ]:
def evaluate(self, board: chess.Board) -> int:
    if board.is_checkmate():
        return -self.VALUE_CHECKMATE
    if board.is_draw():
        return self.VALUE_DRAW
    return None


SimplifiedEvaluation.evaluate = evaluate

Die `SimplifiedEvaluationEngine` Engine integriert die zuvor definierte `SimplifiedEvaluation` Klasse. Gegenüber der vorherigen Implementierung, muss eine Instanz der  `SimplifiedEvaluation` Klasse im Konstruktor erzeugt werden und alle ausgegliederte Funktionalität wie `VALUE_CHECKMATE`, `absolute_heuristic` und `evaluate` über diese Instanz aufgerufen werden.

In [ ]:
class SimplifiedEvaluationEngine(IterativeAlphaBetaCached):

    def __init__(self, max_look_ahead_depth: int):
        super().__init__(max_look_ahead_depth)
        self.evaluator = SimplifiedEvaluation()

    def _evaluate_move(self, board: chess.Board, move: chess.Move, depth: int):
        board.push(move)
        score = self._value(
            board,
            depth,
            -self.evaluator.VALUE_CHECKMATE,
            self.evaluator.VALUE_CHECKMATE
        )
        score *= self.PLAYER_MULTIPLIER[board.turn]
        board.pop()
        return ScoredMove(score=score, move=move)

    @cache_alpha_beta
    def _value(
        self, board: chess.Board, depth: int, alpha: int, beta: int
    ) -> int:
        if (score := self.evaluator.evaluate(board)) is not None:
            return score
        if depth == 0:
            return self.evaluator.absolute_heuristic(board)

        for move in board.legal_moves:
            board.push(move)
            value = -1 * self._value(board, depth - 1, -beta, -alpha)
            board.pop()

            if value >= beta:
                return value
            alpha = max(alpha, value)

        return alpha

Die `SimplifiedEvaluationEngine` kann wie gewohnt getestet werden. Ein Vergleich mit den vorherigen Ergebnissen ergibt an dieser Stelle jedoch keinen Sinn, da die neue Evaluierunsfunktion zu völlig anderen Ergebnissen kommen kann.

In [ ]:
import MinMaxTree

random.seed(42)

engine = SimplifiedEvaluationEngine(max_look_ahead_depth=3)
tree = MinMaxTree.add_tree_to_engine(engine, relative=True)
result_SimplifiedEvaluationEngine = engine.analyse(middlegame_board)

Die jupyter Notebook extension `line_profiler` definiert ein magic command `lprun` mit dem die Performance einer Anweisung analysiert werden kann. In dem Fall ist es von Interesse, welche Anweisungen bei der Analyse eines Brettes mit `engine.analyse(middlegame_board)` am häufigsten aufgerufen werden und am meisten Zeit benötigen. Diese Informationen werden von dem Line Profiler für eine spezifierte Funktion ausgegeben. Am interessanten ist dabei die rekursiv aufgerufene Funktion `_value`, welche es zu optimieren gilt. Der Zugriff erfolgt etwas komplexer mit `SimplifiedEvaluationEngine._value.__wrapped__`, da die Funktion einen decorator hat. 

In [ ]:
%load_ext line_profiler
random.seed(42)

engine = SimplifiedEvaluationEngine(max_look_ahead_depth=3)
%lprun -f SimplifiedEvaluationEngine._value.__wrapped__ engine.analyse(middlegame_board)

Die Auswertung ergibt, dass alle Aufrufe der absoluten Heuristik mit `self.evaluator.absolute_heuristic(board)` ca. 50% der Gesamt Laufzeit ausmachen. An zweiter Stelle und dritter Stelle mit ca. 15% liegt die Evaluierung ` self.evaluator.evaluate(board)` und der Aufruf von `board.push(move)`. Hier sind zunächst keine direkten Optimierungen möglich, da hierfür die Implementierung der `python-chess` Bibliothek optimiert werden müsste. 
Bei der Heuristik dagegen ist es möglich zu optimieren, indem die Auswertung inkrementell erfolgt. Dazu wird eine neue Klasse `SimplifiedIncrementalEvaluation`, welche `SimplifiedEvaluation` erweitert.

In [ ]:
class SimplifiedIncrementalEvaluation(SimplifiedEvaluation):
    pass

Zunächst wird eine `init` Methode definiert, welche ein Schachbrett `board` als Parameter entgegennimmt. Diese Funktioniert initialisiert eine Liste an scores. Zu diesem Zeitpunkt sind keine vorherigen Informationen bekannt, weswegen der erste score mit der absoluten Heuristik berechnet werden muss. Die `init` Funktion wird entsprechend am Anfang einer jeden Analyse aufgerufen.

In [ ]:
def init(self, board: chess.Board):
    self.scores = [self.absolute_heuristic(board)]


SimplifiedIncrementalEvaluation.init = init

Nun soll eine `push` Methode definiert werden, welche immer dann aufgerufen werden soll, wenn ein Spielzug innerhalb der Analyse, d.h. innerhalb der rekursiven `_value` Aufrufe, hinzugefügt wird. Die Aufgabe der Methode ist es basierend auf der Bewertung der vorherigen Stellung, performant eine neue Bewertung für die neue Stellung zu berechnen. 
Die Änderung der Bewertung durch den Zug hängt unter anderem davon ab, welche Figur gezogen ist und ob möglicherweise eine generische Figur dabei geschlagen wurde. Diese Informationen werden nicht in `chess.Move` direkt gespeichert, sondern lassen sich nur mit zusätzlichen Information aus dem Board ableiten. Zusätzlich ist auch die Betrachung von einigen Randfällen nötig wie das Schlagen en passant oder die Promotion. Daher wurde sich entschieden eine Klasse als Abstraktion einzuführen, welche die Informationen berechnet und gebündelt speichert. 

Zunächst wird eine Klasse `PostionedPiece` eingeführt, welche neben der Figur vom Typ `chess.Piece` auch dessen aktuelle Position speichert.

In [ ]:
class PostionedPiece:

    def __init__(self, piece: chess.Piece, file: int, rank: int):
        self.piece = piece
        self.file = file
        self.rank = rank
        self.square = chess.square(file_index=file, rank_index=rank)

Die eigentliche Klasse zur Abstraktion ist nun die `DetailedMove` Klasse, welche im Konstruktor das aktuelle board und den move entgegennimmt. Im Konstruktor werden dann die verschiedenen benötigten Informationen abgespeichert. Dabei wird zunächst die ziehende Figur auf ihrem ursprünglichen Feld als `PostionedPiece` in `self.movedPiece` abgespeichert. Anschließend wird analog die ziehende Funktion auf ihrem Zielfeld in `self.placedPiece` abgespeichert. Hierbei muss beachtet werden, dass durch eine Promotion die Figur sich gegebenfalls geändert hat. Zuletzt wird noch die geschlagende Figur, sofern es eine gibt, in `self.capturedPiece` abgespeichert. Hierbei muss der spezielle Fall betrachtet werden, dass es sich um ein Schlagen en passant handelt. In diesem Fall ergibt sich die Position der schlagenden Figur aus der Reihe von `movedPiece`, aber der Spalte von `placedPiece`.
Außerdem wird eine Funktion `hasCapture` definiert, um zu überprüfen, ob mit diesem Zug eine Figur geschlagen wurde.

In [ ]:
class DetailedMove:

    def __init__(self, board: chess.Board, move: chess.Move):
        self.move = move

        self.movedPiece = PostionedPiece(
            piece=board.piece_at(move.from_square),
            file=chess.square_file(move.from_square),
            rank=chess.square_rank(move.from_square)
        )

        if move.promotion:
            self.placedPiece = PostionedPiece(
                piece=chess.Piece(move.promotion, self.movedPiece.piece.color),
                file=chess.square_file(move.to_square),
                rank=chess.square_rank(move.to_square)
            )
        else:
            self.placedPiece = PostionedPiece(
                piece=self.movedPiece.piece,
                file=chess.square_file(move.to_square),
                rank=chess.square_rank(move.to_square)
            )

        if board.is_en_passant(move):
            self.capturedPiece = PostionedPiece(
                piece=chess.Piece(chess.PAWN, not board.turn),
                file=self.movedPiece.file,
                rank=self.placedPiece.rank
            )
        elif (piece := board.piece_at(move.to_square)) is not None:
            self.capturedPiece = PostionedPiece(
                piece=piece,
                file=self.placedPiece.file,
                rank=self.placedPiece.rank
            )
        else:
            self.capturedPiece = None

    def hasCapture(self):
        return self.capturedPiece is not None

Mit Hilfe der definierten Klassen, kann nun die `push` Methode implementiert werden. Diese nimmt als Parameter einen DetailedMove `move`. Als Ausgangspunkt wird der letze score Eintrag genutzt. Dann wird der Wert von `move.movedPiece` abgezogen und der von `move.placedPiece` hinzuaddiert. Sollte mit dem Zug eine Figur geschlagen werden, wird auch dessen Wert zum score hinzuaddiert. Der neue score wird dann negiert an die Liste `self.scores` angefügt, da die Bewertung ja immmer aus der Perspektive des Spielers am Zug ist und das Vorzeichen daher immer wechseln muss.

In [ ]:
def push(self, move: DetailedMove):
    score = self.scores[-1]

    score -= self._get_piece_square_value(
        move.movedPiece.piece, move.movedPiece.file, move.movedPiece.rank
    )

    score += self._get_piece_square_value(
        move.placedPiece.piece, move.placedPiece.file, move.placedPiece.rank
    )

    if move.hasCapture():
        score += self._get_piece_square_value(
            move.capturedPiece.piece,
            move.capturedPiece.file,
            move.capturedPiece.rank
        )

    self.scores.append(-1 * score)


SimplifiedIncrementalEvaluation.push = push

Neben der `push` Methode gibt es entsprechend dann auch eine `pop` Methode, die immer aufgerufen werden soll, wenn ein Zug wieder rückgängig gemacht worden ist. Die Methode nimmt keine Parameter entgegen und entfernt lediglich den jeweils letzen Eintrag der `self.scores` Liste. Diese kurze und performante Implementierung wurde dadurch ermöglicht, dass eben `push` jeden neuen score in der Liste ablegt und nicht den alten überschreibt. 

In [ ]:
def pop(self):
    self.scores.pop()


SimplifiedIncrementalEvaluation.pop = pop

Zuletzt wird noch eine Methode `getScore` ohne Parameter definiert, welche den aktuellen score, d.h. den letzen Eintrag in der Liste, zurückgibt.

In [ ]:
def getScore(self):
    return self.scores[-1]


SimplifiedIncrementalEvaluation.getScore = getScore

Nun kann auch eine Engine mit der inkrementellen simplified evaluation Funktion erstellt werden. Wie zuvor auch sind nur kleine Anpassungen nötig. Im Konstruktor muss selbstverständlich der evaluation nun mit einem Objekt der Klasse `SimplifiedIncrementalEvaluation` initialisiert werden. Für die `_evaluate_move` Methode wurde nur die Zeile `self.evaluator.init(board)` hinzugefügt. Diese dient dazu das Board einmal zu Beginn mit der absoluten Heuristik auszuwerten, sodass dann anschließend folgende Positionen inkrementell berechnet werden können. In der `value` Methode muss immer vor dem pushen eines neues Zuges auf das Board `self.evaluator.push(DetailedMove(board, move))` und beim entfernen eines Zuges `self.evaluator.pop()` aufgerufen werden. 

In [ ]:
class SimplifiedIncrementalEvaluationEngine(SimplifiedEvaluationEngine):

    def __init__(self, max_look_ahead_depth: int):
        super().__init__(max_look_ahead_depth)
        self.evaluator = SimplifiedIncrementalEvaluation()

    def _evaluate_move(self, board: chess.Board, move: chess.Move, depth: int):
        board.push(move)
        self.evaluator.init(board)

        score = self._value(
            board,
            depth,
            -self.evaluator.VALUE_CHECKMATE,
            self.evaluator.VALUE_CHECKMATE
        )
        score *= self.PLAYER_MULTIPLIER[board.turn]
        board.pop()
        return ScoredMove(score=score, move=move)

    @cache_alpha_beta
    def _value(
        self, board: chess.Board, depth: int, alpha: int, beta: int
    ) -> int:
        if score := self._evaluate(board):
            return score
        if depth == 0:
            return self.evaluator.getScore()

        for move in board.legal_moves:
            detailedMove = DetailedMove(board, move)
            self.evaluator.push(detailedMove)
            board.push(move)

            value = -1 * self._value(board, depth - 1, -beta, -alpha)
            self.evaluator.pop()
            board.pop()

            if value >= beta:
                return value
            alpha = max(alpha, value)

        return alpha

Die definierte Engine kann getestet werden und in diesem Fall auch wieder das Ergebnis mit dem vorherigen verglichen werden, da die inkrementelle Bewertung die Ergebnisse nicht beeinflussen sollte.

In [ ]:
random.seed(42)

engine = SimplifiedIncrementalEvaluationEngine(max_look_ahead_depth=3)
tree = MinMaxTree.add_tree_to_engine(engine, relative=True)
result_SimplifiedIncrementalEvaluationEngine = engine.analyse(middlegame_board)

assert result_SimplifiedIncrementalEvaluationEngine == result_SimplifiedEvaluationEngine

Wieder kann mit dem Line Profiler die `_value` Methode analysiert werden. Zu sehen ist, dass die eingefügten Änderungen zusammen nur noch ca. 20% der Gesamtzeit ausmachen. Dies stellt eine deutliche Verbesserung gegenüber den vorherigen Implementierung mit 50% dar. 

In [ ]:
%load_ext line_profiler
random.seed(42)

engine = SimplifiedIncrementalEvaluationEngine(max_look_ahead_depth=3)
%lprun -f SimplifiedIncrementalEvaluationEngine._value.__wrapped__ engine.analyse(middlegame_board)

## Quiescence Search

Ein aktuelles Problem der Engine ist, dass sie immer bei einer fixen Tiefe aufhört zu analysieren unabhängig von der Stellung. Dies kann dazu führen, dass die Engine in zum Beispiel einem Schlagabtausch die Suche beendet und somit die Stellung völlig falsch interpretiert wird. Oder die Engine versucht eine für sie ungünstige Situation durch z.B. das Geben von Schach soweit hinauszugöern, dass diese Situation in der limitierten Suchtiefe nicht mehr auftreten kann. Hierbei spricht man auch von dem [Horizon Effekt](https://www.chessprogramming.org/Horizon_Effect). 
Um dies zu vermeiden, ist die Idee, nur ruhige Stellungen mit der Heuristik zu analysieren. Als ruhige Stellung zählen solche in denen keine Figur mehr geschlagen werden kann und je nach Implementierung auch kein Schach gegeben werden kann. In den anderen Fällen wird eine eingeschränkte Suche gemacht, die genau solche Züge noch weiter analysiert. Dies wird [quiescence search](https://www.chessprogramming.org/Quiescence_Search) genannt.
Dazu wird nun eine neue Klasse `QuiescenceEngine`, welche die vorherige Engine `SimplifiedIncrementalEvaluationEngine` erweitert.

In [ ]:
class QuiescenceEngine(SimplifiedIncrementalEvaluationEngine):
    pass

An der `_value` Funktion muss lediglich eine Anpassung gemacht werden. Im dem Fall, dass die Tiefe 0 ist wird nun die Funktion `_quiescence` aufgerufen.

In [ ]:
@cache_alpha_beta
def _value(self, board: chess.Board, depth: int, alpha: int, beta: int) -> int:
    if score := self._evaluate(board):
        return score
    if depth == 0:
        return self._quiescence(board, alpha, beta)

    for move in board.legal_moves:
        detailedMove = DetailedMove(board, move)

        self.evaluator.push(detailedMove)
        board.push(move)
        value = -1 * self._value(board, depth - 1, -beta, -alpha)
        board.pop()
        self.evaluator.pop()

        if value >= beta:
            return value
        alpha = max(alpha, value)

    return alpha


QuiescenceEngine._value = _value

Die `_quiesce` Methode nimmt als Parameter ein `chess.Board` board, einen `int` alpha und einen `int` beta und gibt genauso wie die `_value` Funktion den score der aktuellen Position zurück. Zunächst wird ein Standing Pat berechnet, dabei handelt es sich um die statische Evaluation der aktuellen Stellung. Ist diese höher als beta, kann direkt beta zurückgegeben werden. Ansonsten wird `alpha` angehoben, sofern der Wert größer ist. `alpha` wird auch mindestens von der Funktion zurückgegeben, sodass gilt `stand_path <= alpha <= _quiesce(board, alpha, beta) <= beta`. 

Andernfalls ist der Algorithmus vergleichbar zu dem der `_value` Funktion. Es gibt jedoch zwei wesentliche Unterschiede. Einerseits werden hier nicht alle möglichen Züge betrachtet, sondern nur solche, die Figuren schlagen. Der andere Unterschied ist, dass bevor die restliche Implementierung von NegaMax folgt, noch einige Optimierungen erfolgen. 
Diese sind notwendig, denn ein großer Teil der Suchzeit nimmt die quiescence search ein, da in nahezu jeder Position einige Schlagabtäusche möglich sind und die Anzahl der Knoten damit massiv ansteigt. 

Konkret werden Züge nicht analysiert sofern keine Promotion vorliegt und DeltaPruning erfolgt oder die Static Exchange Evaluation kleiner 0 ist. Beide Optimierungen werden nachfolgend noch geklärt. Andenfalls wird wie bereits aus dem NegaMax Algorithmus bekannt fortgefahren.

In [ ]:
def _quiescence(self, board: chess.Board, alpha: int, beta: int) -> int:
    stand_pat = self.evaluator.getScore()

    if stand_pat >= beta:
        return beta
    alpha = max(alpha, stand_pat)

    for move in board.generate_legal_captures():
        detailedMove = DetailedMove(board, move)

        if move.promotion is None:
            if self._canDeltaPrune(stand_pat, alpha, detailedMove):
                continue
            if self._seeCapture(board, detailedMove) < 0:
                continue

        board.push(move)
        self.evaluator.push(detailedMove)

        value = -1 * self._quiescence(board, -beta, -alpha)

        self.evaluator.pop()
        board.pop()

        if value >= beta:
            return value
        alpha = max(alpha, value)
    return alpha


QuiescenceEngine._quiescence = _quiescence

Die Idee von Delta Pruning ist, dass es einige Stellungen gibt in denen der Spieler bereits soweit zurückliegt, dass auch potentielle gute Züge nicht mehr helfen können. Ein Beispiel hierfür ist, wenn der Spieler eine Dame zurückliegt, dann ändert das Schlagen eines Bauerns oder einer Leichtfigur nichts daran, dass er immer noch zurückliegt. 
Die Methode `_canDeltaPrune` überprüft genau dies und nimmt den standing pat, alpha und den zu testenden move als Parameter entgegen. Kann der Zug durch delta Pruning ignoriert werden, gibt die Methode `True` zurück, andernfalls `False`. 
Zunächst berechnet sie zum aktuellen Stand Pat den Wert der zu schlagenden Figur hinzu, sowie einen potentiellen Stellungsvorteil von 2 Bauern, daher 200 Centipawns. Hierbei handelt es sich gewissermaßen, um eine sehr optimistische Schätzung für den Spieler. Denn dieser bekommt einen Positionsvorteil zugesprochen und es wird davon ausgegangen, dass er die Figur einfach Schlagen kann ohne zurückgeschlagen zu werden. Liegt dieser optmistische Wert trotzdem unter `alpha`, so lohnen sich weitere Untersuchungen nicht. In diesem Fall gibt die Methode `True` zurück, andernfalls `False`. 

In [ ]:
def _canDeltaPrune(
    self, stand_pat: int, alpha: int, detailedMove: DetailedMove
):
    # TODO: Don't do this in endgames
    pieceValue = self.evaluator.PIECE_VALUES[
        detailedMove.capturedPiece.piece.piece_type]
    bestAlpha = stand_pat + pieceValue + 200
    return bestAlpha < alpha


QuiescenceEngine._canDeltaPrune = _canDeltaPrune

Die zweite Optimierung `Static Exchange Evaluation` analysiert den potentiellen Schlagabtausch zwischen den Spielern. Dabei wird angenommen, dass beide Spieler jeweils mit ihrer geringwertigsten Figur die Figur des Gegners schlagen, sofern am Ende des Schlagabtausches einen Vorteil für den jeweiligen Spieler entsteht. 
Die Methode `_seeCapture` nimmt als Parameter das aktuelle board und den Zug entgegen und gibt den Wert des Schlagabtausches zurück. Ein positver Wert bedeutet dabei, dass der Spieler am Zug durch den Schlagabtausch vorraussichtlich Material gewinnt und ein negativer, dass dieser vorraussichtlich Material verliert. Daher werden die Züge, welche einen Wert kleiner 0 haben, in der `_quiescence` Methode übersprungen. 
Die Methode pusht zunächst den übergeben Zug, berechnet den Static Exchange Evaluation score, entfernt den Zug wieder und gibt den score zurück. Die Berechnung des Scores ist grundsätzlich rekursiv implementiert. Dabei wird immer der Static Exchange Evaluation score des verbleibendenen Schlagabtausches von dem Wert der eben geschlagenden Figur abgezogen. Die Rekursion findet dann allerdings in der Methode `_see` statt, da für den verbleibdenen Schlagabtausch die Züge erst berechnet werden müssen.

In [ ]:
def _seeCapture(self, board: chess.Board, detailedMove: DetailedMove):
    board.push(detailedMove.move)
    capturedPiece = detailedMove.capturedPiece
    value = self.evaluator.PIECE_VALUES[
        capturedPiece.piece.piece_type
    ] - self._see(board, capturedPiece.square)
    board.pop()
    return value


QuiescenceEngine._seeCapture = _seeCapture

Die `_see` Methode bekommt ebenfalls das aktuelle Board als Parameter, sowie nun das Schachfeld auf dem der Schlagabtausch stattfindet und gibt den Static Exchange Evaluation Score zurück. Zunächst werden alle Angreifer des Felder mit der `attackers` Methode berechnet. Dabei müssen pinned Figuren ignoriert werden mit der `is_pinned` Methode. Wenn es keinen Attacker gibt, wird 0 zurückgegeben. Andernfalls wird die Figur mit dem geringsten Wert ausgewählt und der entsprechende Zug ausgeführt. Dann wir der score wieder rekursiv berechnet, indem der Score des verbleibenden Schlagabtausches  von der eben geschlagenen Figur abgezogen wird. Ist der score niedriger als 0, wird er auf 0 gesetzt. Der Grund hierfür ist, dass der Schlagabtausch vom Spieler nur durchgeführt werden, wenn dies für ihn vorteilhaft ist. Anschließend wird der eben gespielte Zug wieder entfern und der score zurückgegeben.

In [ ]:
def _see(self, board: chess.Board, square: chess.Square):
    attackers_squares = [
        square for square in board.attackers(board.turn, square)
        if not board.is_pinned(board.turn, square)
    ]
    if not attackers_squares:
        return 0

    attacker_square = min(
        attackers_squares,
        key=lambda attackers_square: board.piece_type_at(attackers_square)
    )
    capturedPiece = board.piece_type_at(square)

    board.push(chess.Move(from_square=attacker_square, to_square=square))
    value = max(
        0,
        self.evaluator.PIECE_VALUES[capturedPiece] - self._see(board, square)
    )
    board.pop()
    return value


QuiescenceEngine._see = _see

Die Funktion wird anschließend getestet mit einigen Beispielen. Im ersten Test ist einer der Angreifer gepinned und kann daher nicht schlagen.

In [ ]:
board = chess.Board("6k1/6b1/8/4p3/3P4/8/8/1K4R1 w - - 0 1")
IPython.display.display(board)
engine = QuiescenceEngine(max_look_ahead_depth=3)
result = engine._see(board, chess.E5)
assert result == 100

Im zweiten Beispiel sind nicht alle Schlagabtäusche sinnvoll, daher soll hier getestet werden, ob dies entsprechend bei der Bewertung berücksichtigt wird.

In [ ]:
board = chess.Board("6k1/6b1/5p2/4q3/3P4/8/8/1K2Q3 w - - 0 1")
IPython.display.display(board)
engine = QuiescenceEngine(max_look_ahead_depth=3)
result = engine._see(board, chess.E5)
assert result == 800

In [ ]:
random.seed(42)
board = chess.Board("8/1R6/4Q3/8/3k1B2/5R2/P1P3PP/6K1 w - - 7 95")
engine = QuiescenceEngine(max_look_ahead_depth=2)
engine.analyse(board)

Die eben definierte Engine kann nun einmal komplett getestet werden auf dem üblichen `middlegame_board`.

In [ ]:
random.seed(42)

engine = QuiescenceEngine(max_look_ahead_depth=3)
engine.analyse(middlegame_board)

In [ ]:
random.seed(42)

engine = QuiescenceEngine(max_look_ahead_depth=3)
tree = MinMaxTree.add_tree_to_engine(engine, relative=True)
engine.analyse(middlegame_board)

tree.count(quiesce=True)

In [ ]:
%lprun -f QuiescenceEngine._quiescence  engine.analyse(middlegame_board)

In [ ]:
%lprun -f QuiescenceEngine._see  engine.analyse(middlegame_board)